In [19]:
import numpy as np
import pandas as  pd
from camo import greedy
from matplotlib import pyplot as plot

In [18]:
paysim = pd.read_csv("PS_20174392719_1491204439457_log.csv")
paysim.head()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [70]:
paysim['nameOrig'].value_counts()
max(paysim["step"])


743

In [63]:
transaction = paysim[["nameOrig", "nameDest"]]
transaction = transaction.join(transaction, on="nameDest", rsuffix="h")
transaction.head()

,nameOrig,nameDest,nameOrigh,nameDesth
0,C1231006815,M1979787155,NaN,NaN
1,C1666544295,M2044282225,NaN,NaN
2,C1305486145,C553264065,NaN,NaN
3,C840083671,C38997010,NaN,NaN
4,C2048537720,M1230701703,NaN,NaN


In [37]:
indexer = dict()
counter =0
for i in transaction.nameOrig.unique():
    indexer[i] = counter
    counter +=1

counter *= 2
for i in transaction.nameDest.unique():
    indexer[i] = counter
    counter+=1

In [45]:
remapped_orig = []
remapped_dest = []

for row in transaction.iterrows():
    row = row[1]
    remapped_orig.append(indexer[row["nameOrig"]])
    remapped_dest.append(indexer[row["nameDest"]])

In [46]:
M = greedy.listToSparseMatrix(remapped_orig, remapped_dest)
lwRes = greedy.logWeightedAveDegree(M)

finished computing weight matrix
finished initialization
finished setting deltas
finished building min trees
current set size = 30800000
current set size = 30700000
current set size = 30600000
current set size = 30500000
current set size = 30400000
current set size = 30300000
current set size = 30200000
current set size = 30100000
current set size = 30000000
current set size = 29900000
current set size = 29800000
current set size = 29700000
current set size = 29600000
current set size = 29500000
current set size = 29400000
current set size = 29300000
current set size = 29200000
current set size = 29100000
current set size = 29000000
current set size = 28900000
current set size = 28800000
current set size = 28700000
current set size = 28600000
current set size = 28500000
current set size = 28400000
current set size = 28300000
current set size = 28200000
current set size = 28100000
current set size = 28000000
current set size = 27900000
current set size = 27800000
current set size = 2770

current set size = 1600000
current set size = 1500000
current set size = 1400000
current set size = 1300000
current set size = 1200000
current set size = 1100000
current set size = 1000000
current set size = 900000
current set size = 800000
current set size = 700000
current set size = 600000
current set size = 500000
current set size = 400000
current set size = 300000
current set size = 200000
current set size = 100000


In [54]:
fraud_participants  = list(set(lwRes[0][1]).union( set(lwRes[0][0])))

In [56]:
fraud_mark = dict()

for row in paysim.iterrows():
    row = row[1]
    if row["isFraud"]:
        fraud_mark[indexer[row["nameOrig"]]]=1
        fraud_mark[indexer[row["nameDest"]]]=1

In [57]:
f_count = 0
for i in fraud_participants:
    f_count += fraud_mark.get(i, 0)
    

In [58]:
f_count

6